In [1]:
!wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
!unzip wwm_uncased_L-24_H-1024_A-16.zip

!ls wwm_uncased_L-24_H-1024_A-16

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

!ls uncased_L-12_H-768_A-12

!pip install simpletransformers
!pip install transformers
!pip install tensorboardx
!pip install ktrain

--2020-07-02 05:45:55--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip.3’

wwm_uncased_L-24_H- 100%[===================>]   1.16G  75.8MB/s    in 16s     

2020-07-02 05:46:12 (72.6 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip.3’ saved [1248381879/1248381879]

Archive:  wwm_uncased_L-24_H-1024_A-16.zip
replace wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta
--2020-07-02 05:48:28--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-

In [2]:
!pip install bert
!pip install bert-tensorflow
!pip install tensorflow_hub
!pip install bert-for-tf2

  Created wheel for bert: filename=bert-2.2.0-cp36-none-any.whl size=2781 sha256=a139ff3a3cd37646a034d9a7e54ab8ff5d397ddc5ad0a0e0c3a71e6634e801dc
  Stored in directory: /home/ec2-user/.cache/pip/wheels/fe/71/b7/941459453bd38e5d97a8c886361dee19325e9933c9cf88ad46
  Created wheel for erlastic: filename=erlastic-2.0.0-cp36-none-any.whl size=5790 sha256=fae2684614eed46ce2b57145cc35d6fc1480839c676d3bef22833730cd6cf9cd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/02/62/46/93c713a5f061aeeb4f16eb6bf5ee798816e6ddda70faa78e69
Successfully built bert erlastic
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 71kB 3.4MB/s eta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 102kB 3.9MB/s ta 0:00:011
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 40kB 3.3MB/s eta 0:00:011
  Created wh

In [13]:
import pandas as pd
import re
import string
import bert
import tensorflow_hub as hub
import numpy as np
from sklearn.model_selection import train_test_split
import os
#from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
import tensorflow as tf
from tensorflow import keras as keras
from tensorflow.keras import layers
import tqdm
from tensorflow.keras import layers
import pickle
import math
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [14]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [3]:
def tokenize_claims(text_claims):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_claims))

In [15]:
def create_model(max_seq_len,cnn_filters,dropout_rate,dnn_units):
    input_ids = keras.layers.Input(
                                   shape=(max_seq_len, ),
                                   dtype='int32',
                                   name="input_ids"
                                   )
    input_mask = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_mask")
    segment_ids = keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32, name="segment_ids")
    
    _, bert_output = bert_layer([input_ids, input_mask, segment_ids])
    #bert_output = bert_layer(input_ids)
    print("bert shape", bert_output.shape)
    
    
    cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer1 = layers.GlobalMaxPool1D()(cnn_layer1)
    cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer2 = layers.GlobalMaxPool1D()(cnn_layer2)
    cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")(bert_output)
    cnn_layer3 = layers.GlobalMaxPool1D()(cnn_layer3)

    concatenated = tf.concat([cnn_layer1, cnn_layer2, cnn_layer3], axis=-1) # (batch_size, 3 * cnn_filters)   
    dense_1 = layers.Dense(units=dnn_units, activation="relu")(concatenated)
    dropout = layers.Dropout(rate=dropout_rate)(concatenated)
       
    last_dense = layers.Dense(units=1,activation="sigmoid")(dropout)
    
    model = keras.Model(inputs=[input_ids,input_mask,segment_ids], outputs=last_dense)
    
    #cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    #cls_out = keras.layers.Dropout(0.5)(cls_out)
    #logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
    #logits = keras.layers.Dropout(0.5)(logits)
    #logits = keras.layers.Dense(
    #units=len(classes),
    #activation="softmax"
    #)(logits)
    #model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))
    #load_stock_weights(bert, bert_ckpt_file)
    return model

In [16]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
bucket='trainedbertmodelweights'
data_key = 'BERT_EMBEDDINGS_TRAINABLE_CNN_weights-improvement-19-0.98.hdf5'
weights_location = 's3://{}/{}'.format(bucket, data_key)
print(weights_location)

s3_client = boto3.client('s3')
s3_client.download_file(bucket, data_key, 'hello2.txt')

s3://trainedbertmodelweights/BERT_EMBEDDINGS_TRAINABLE_CNN_weights-improvement-19-0.98.hdf5


In [17]:
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have","she's":"she is","he's":"he is"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

def preprocess_text(sen):
    #convert ’ to '
    
    sentence = sen.replace("’","'")
    
    #expand contractions 
    sentence = expand_contractions(sentence.lower())
    # Removing punctuation
    #sentence = re.sub('<[^>]+>',' ', sen)

    # Remove punctuations and numbers and foreign characters
    sentence = re.sub('[^a-zA-Z]',' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+",' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+',' ', sentence)

    return sentence

In [18]:
precision_obj = tf.keras.metrics.Precision()
recall_obj = tf.keras.metrics.Recall()
#Get model for Experiment3 created
CNN_FILTERS = 200
DROPOUT_RATE = 0.1
DNN_UNITS = 128
model = create_model(60,CNN_FILTERS,DROPOUT_RATE,DNN_UNITS)
#load model weights
model.load_weights("./weights.hdf5")

model.compile(
               optimizer=keras.optimizers.Adam(1e-5),
               loss='binary_crossentropy',
               metrics=['accuracy',precision_obj,recall_obj]
             )

bert shape (None, None, 768)


In [19]:
def predict_claims(claim_text):
    claims = [preprocess_text(claim_text)]

    max_seq_len=60
    x_input,masks,segments = [], [],[]
    for new_claim in claims:
        text = tokenizer.tokenize(new_claim)
        text = text[:max_seq_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_seq_len - len(input_sequence)
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_seq_len
        x_input.append(np.array(tokens))
        masks.append(np.array(pad_masks))
        segments.append(np.array(segment_ids))

    return model.predict(x = [x_input,masks,segments])[0]

In [20]:
predict_claims('masks spread the coronavirus')[0]

0.00062717195

In [12]:
!pip3 freeze > requirements.txt